In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import yfinance as yf

df = yf.download("AAPL", start="2026-01-07", interval='1m')
df = df[["Open", "High", "Low", "Close", "Volume"]].dropna()
# df.head()

if isinstance(df.columns, pd.MultiIndex):
    df.columns = df.columns.get_level_values(0)
df.head()

In [ ]:
def add_candle_features(df: pd.DataFrame) -> pd.DataFrame:
  df = df.copy()
  o, h, l, c = df["Open"], df["High"], df["Low"], df["Close"]
  rng = (h-l).replace(0, np.nan)

  df["Range"] = rng
  df["Body"] = (c-o).abs()
  df["Direction"] = np.sign(c-o)
  df["UpperWick"] = h-np.minimum(c, o)
  df["LowerWick"] = np.minimum(c, o)-l
  df["UpperWickPct"] = df["UpperWick"]/rng
  df["LowerWickPct"] = df["LowerWick"]/rng
  df["BodyPct"] = df["body"]/rng

  return df

In [ ]:
def bullish_engulfing(df: pd.DataFrame) -> pd.Series:
    o, c = df["Open"], df["Close"]
    o1, c1 = o.shift(1), c.shift(1)

    prev_bear = c1 < o1
    curr_bull = c > o
    engulf = (o <= c1) & (c >= o1)

    return (prev_bear & curr_bull & engulf).fillna(False)


def hammer(df: pd.DataFrame) -> pd.Series:
    return (
        (df["BodyPct"] <= 0.35) &
        (df["LowerWickPct"] >= 0.55) &
        (df["UpperWickPct"] <= 0.15)
    ).fillna(False)


def inside_bar(df: pd.DataFrame) -> pd.Series:
    return (
        (df["High"] < df["High"].shift(1)) &
        (df["Low"] > df["Low"].shift(1))
    ).fillna(False)


def test_pattern(
    df_raw: pd.DataFrame,
    pattern_fn,
    horizons=(1, 3, 5),
    side="long"
) -> pd.DataFrame:
    df = add_candle_features(df_raw)
    sig = pattern_fn(df)

    entry = df["Open"].shift(-1)
